In [12]:
import pandas as pd
import numpy as np

# Tabular operations in `pandas`
We've already seen a simple _table join_ operation when we used the SA1 codes as an index to select only SA1 data for the Wellington urban area. In this notebook we look at the various ways that tables can be combined in `pandas` a bit more closely.

## Joining tables with `merge()`
As an example here are data pertaining to airports sourced via https://ourairports.com/data/.

In [13]:
airports_all = pd.read_csv(
    "https://davidmegginson.github.io/ourairports-data/airports.csv")

Let's cut things down so we are only dealing with airports in New Zealand that are significant enough to have an IATA code, and also slim the data down to only columns we really care about.

In [14]:
airports_nz = airports_all[(airports_all.iso_country == "NZ") &
                        (airports_all.iata_code.notna())]
airports_nz = airports_nz[
    ["name", "latitude_deg", "longitude_deg", "iata_code"]]
airports_nz = airports_nz \
    .rename(columns = {"latitude_deg": "lat", "longitude_deg": "lon"})
airports_nz

,name,lat,lon,iata_code
50060,Fox Glacier Aerodrome,-43.461110,170.017020,FGL
50124,Auckland International Airport,-37.011990,174.786331,AKL
50129,Taupo Airport,-38.739700,176.084000,TUO
50130,Ardmore Airport,-37.029701,174.973007,AMZ
50131,Ashburton Airport,-43.903301,171.796997,ASG
50138,Christchurch International Airport,-43.489029,172.532065,CHC
50139,Chatham Islands / Tuuta Airport,-43.811890,-176.465140,CHT
50141,Coromandel Airport,-36.791698,175.509003,CMV
50142,Dargaville Aerodrome,-35.939701,173.893997,DGR
50145,Dunedin International Airport,-45.928101,170.197998,DUD


Rather out of date (but free!) information about scheduled flights is available via https://openflights.org/data.php. Per the information on that page, there are no column names in the data, so we have to tell `read_csv()` that using `header = None` and then cut the data down to the columns we want (the origin and destination IATA codes) and name them.

In [15]:
schedule = pd.read_csv("https://raw.githubusercontent.com/jpatokal/openflights/master/data/routes.dat", header = None)
schedule = schedule.iloc[:, [2, 4]]
schedule.columns = ["iata_code_1", "iata_code_2"]
schedule

,iata_code_1,iata_code_2
0,AER,KZN
1,ASF,KZN
2,ASF,MRV
3,CEK,KZN
4,CEK,OVB
...,...,...
67658,WYA,ADL
67659,DME,FRU
67660,FRU,DME
67661,FRU,OSS


Now we want to associate with each scheduled flight within New Zealand the latitude-longitude of its respective airports.

We do such a table join using the `DataFrame.merge()` method. There are a few ways to approach this. We'll see the cleanest approach at the end of this section. For now we'll wander through some messier approaches to show how the `merge` function works.

We have to tell the `merge` function which column names we want to perform the join using.

In [16]:
schedule.merge(airports_nz, left_on = "iata_code_1", right_on = "iata_code")

,iata_code_1,iata_code_2,name,lat,lon,iata_code
0,AKL,BNE,Auckland International Airport,-37.011990,174.786331,AKL
1,AKL,PPT,Auckland International Airport,-37.011990,174.786331,AKL
2,AKL,SYD,Auckland International Airport,-37.011990,174.786331,AKL
3,CHC,SYD,Christchurch International Airport,-43.489029,172.532065,CHC
4,AKL,PVG,Auckland International Airport,-37.011990,174.786331,AKL
...,...,...,...,...,...,...
213,CHC,OOL,Christchurch International Airport,-43.489029,172.532065,CHC
214,CHC,SYD,Christchurch International Airport,-43.489029,172.532065,CHC
215,WLG,BNE,Wellington International Airport,-41.327202,174.804993,WLG
216,WLG,MEL,Wellington International Airport,-41.327202,174.804993,WLG


The result of this is what we want alright. The `lat` and `lon` attributes for the first airport in each case have been joined as we wanted. Now if we want to join the second airport code we can do the same again, but setting `iata_code_2` as the `left_on1` option:

In [17]:
schedule \
    .merge(airports_nz, left_on = "iata_code_1", right_on = "iata_code") \
    .merge(airports_nz, left_on = "iata_code_2", right_on = "iata_code")

,iata_code_1,iata_code_2,name_x,lat_x,lon_x,iata_code_x,name_y,lat_y,lon_y,iata_code_y
0,AKL,CHC,Auckland International Airport,-37.011990,174.786331,AKL,Christchurch International Airport,-43.489029,172.532065,CHC
1,AKL,DUD,Auckland International Airport,-37.011990,174.786331,AKL,Dunedin International Airport,-45.928101,170.197998,DUD
2,AKL,WLG,Auckland International Airport,-37.011990,174.786331,AKL,Wellington International Airport,-41.327202,174.804993,WLG
3,AKL,ZQN,Auckland International Airport,-37.011990,174.786331,AKL,Queenstown International Airport,-45.021099,168.738998,ZQN
4,CHC,AKL,Christchurch International Airport,-43.489029,172.532065,CHC,Auckland International Airport,-37.011990,174.786331,AKL
...,...,...,...,...,...,...,...,...,...,...
109,WRE,WLG,Whangarei Airport,-35.769253,174.363713,WRE,Wellington International Airport,-41.327202,174.804993,WLG
110,WSZ,WLG,Westport Airport,-41.737111,171.579033,WSZ,Wellington International Airport,-41.327202,174.804993,WLG
111,ZQN,AKL,Queenstown International Airport,-45.021099,168.738998,ZQN,Auckland International Airport,-37.011990,174.786331,AKL
112,ZQN,CHC,Queenstown International Airport,-45.021099,168.738998,ZQN,Christchurch International Airport,-43.489029,172.532065,CHC


Things are now a bit messy because `pandas` tries to resolve issues with duplicate column names by adding default name suffixes `_x` and `_y`. That works OK, but can quickly get rather confusing. There are options in the `merge` function to specify different suffixes which we might want to try.

In [18]:
schedule \
    .merge(airports_nz, left_on = "iata_code_1", right_on = "iata_code") \
    .merge(airports_nz, left_on = "iata_code_2", right_on = "iata_code", 
           suffixes = ["_orig", "_dest"])

,iata_code_1,iata_code_2,name_orig,lat_orig,lon_orig,iata_code_orig,name_dest,lat_dest,lon_dest,iata_code_dest
0,AKL,CHC,Auckland International Airport,-37.011990,174.786331,AKL,Christchurch International Airport,-43.489029,172.532065,CHC
1,AKL,DUD,Auckland International Airport,-37.011990,174.786331,AKL,Dunedin International Airport,-45.928101,170.197998,DUD
2,AKL,WLG,Auckland International Airport,-37.011990,174.786331,AKL,Wellington International Airport,-41.327202,174.804993,WLG
3,AKL,ZQN,Auckland International Airport,-37.011990,174.786331,AKL,Queenstown International Airport,-45.021099,168.738998,ZQN
4,CHC,AKL,Christchurch International Airport,-43.489029,172.532065,CHC,Auckland International Airport,-37.011990,174.786331,AKL
...,...,...,...,...,...,...,...,...,...,...
109,WRE,WLG,Whangarei Airport,-35.769253,174.363713,WRE,Wellington International Airport,-41.327202,174.804993,WLG
110,WSZ,WLG,Westport Airport,-41.737111,171.579033,WSZ,Wellington International Airport,-41.327202,174.804993,WLG
111,ZQN,AKL,Queenstown International Airport,-45.021099,168.738998,ZQN,Auckland International Airport,-37.011990,174.786331,AKL
112,ZQN,CHC,Queenstown International Airport,-45.021099,168.738998,ZQN,Christchurch International Airport,-43.489029,172.532065,CHC


That's a little bit nicer, although really, we would probably want to do quite a lot of dropping of duplicate columns and renaming. The promised much cleaner approach is below.

In [19]:
nz_scheduled_flights = schedule \
    .merge(airports_nz.add_suffix("_1")) \
    .merge(airports_nz.add_suffix("_2")) \
    .drop(columns = ["name_1", "name_2"])
nz_scheduled_flights

,iata_code_1,iata_code_2,lat_1,lon_1,lat_2,lon_2
0,AKL,CHC,-37.011990,174.786331,-43.489029,172.532065
1,AKL,DUD,-37.011990,174.786331,-45.928101,170.197998
2,AKL,WLG,-37.011990,174.786331,-41.327202,174.804993
3,AKL,ZQN,-37.011990,174.786331,-45.021099,168.738998
4,CHC,AKL,-43.489029,172.532065,-37.011990,174.786331
...,...,...,...,...,...,...
109,WRE,WLG,-35.769253,174.363713,-41.327202,174.804993
110,WSZ,WLG,-41.737111,171.579033,-41.327202,174.804993
111,ZQN,AKL,-45.021099,168.738998,-37.011990,174.786331
112,ZQN,CHC,-45.021099,168.738998,-43.489029,172.532065


`add_suffix()` adds `"_"` to all the column names in `airports` before the merge operation. Because that will make `iata_code` into `iata_code_1` which matches the name of column on which we want to base the join in the `schedule` table we no longer have to specify `left_on` and `right_on` (`pandas` finds the matching names and uses them) or worry about suffixes. The same thing happens in the second merge operation. An additional benefit of this approach is that after the join has happened we retain only one copy of the `iata_code_*` columns.

With careful consideration of column names it is often possible to perform table joins like this more cleanly than if you just charge ahead without much forethought!

### Left, right, inner, outer, and cross joins
By default `merge()` performs an **inner** join. This means that only rows where a match was found between the two tables will be retained. That's why the `schedule` data got cut down to size from an initial 67,663 routes to only 114 routes internal to New Zealand. After the first join only 218 routes remain in the data (only 218 scheduled routes had `iata_code_1` matching a New Zealand airport IATA code in the `airports` data). The second join slimmed this down further to only 114 routes where both codes are matched.

An **outer** join retains all rows from both tables regardless of matches, while **left** and **right** joins respectively prioritise retention of rows from the left or right tables in the join. A **cross** join includes output for every pairwise combination of the left and right column values on which the join is based (it's extremely unlikely that you want to do this).

We can see these outcomes reflected in the size of the tables that result from the different options which we set using the `how` parameter. This is easier to follow if we only apply the first join.

In [20]:
methods = ["inner", "outer", "left", "right", "cross"]
for method in methods:
    print(f"{method}: {schedule.merge(airports_nz.add_suffix('_1'), how = method).shape}")

inner: (218, 5)
outer: (67697, 5)
left: (67663, 5)
right: (252, 5)
cross: (3992117, 6)


In this case the default `inner` join is exactly what we want. You may often want a `how = "left"` to prioritise retaining all records in your original dataset, even if no match is found in the data to be joined. The other options are less likely to be useful but it is good to be aware of them! 

## Concatenating with `concat()`
Sometimes you have data from more than one source that are organised identically&mdash;in particular they have the same or shared column names. Less often you may have data from two different sources that record different attributes for the same set of objects, arranged in the same order. In either of these cases you can basically 'sticky-tape' the tables or series together using `concat`. We already saw this in action [back here](02-navigating-pandas.ipynb#dataframe). As usual, you can combine data row-wise or column-wise. When you want to make up a `DataFrame` from a bunch of `Series` you do `pd.concat(<list of Series>, axis = "columns")` but sometimes you just want to extend `Series` by appending them to others.

In [21]:
s1 = pd.Series(range(5), index = list("abcde"))
s2 = pd.Series(range(6, 11), index = list("fghij"))
print(pd.concat([s1, s2]))

a     0
b     1
c     2
d     3
e     4
f     6
g     7
h     8
i     9
j    10
dtype: int64


If we concatenate these two series by columns, there will be many missing values:

In [22]:
print(pd.concat([s1, s2], axis = "columns"))

     0     1
a  0.0   NaN
b  1.0   NaN
c  2.0   NaN
d  3.0   NaN
e  4.0   NaN
f  NaN   6.0
g  NaN   7.0
h  NaN   8.0
i  NaN   9.0
j  NaN  10.0


The most likely use for this method is if you have data for a number of different places organised in similar tables and you want to combine them into single table. For example, say we had New Zealand and Australia airport data. We already have the New Zealand airports in the `airports_nz` dataframe. Say we also have an `airports_au` dataset

In [23]:
airports_au = airports_all[(airports_all.iso_country == "AU") &
                           (airports_all.iata_code.notna())]
airports_au = airports_au[
   ["name", "iata_code", "latitude_deg", "longitude_deg"]] \
   .rename(columns = {"latitude_deg": "lat", "longitude_deg": "lon"})
airports_au

,name,iata_code,lat,lon
9918,Angus Downs Airport,ANZ,-25.032500,132.274800
11039,Lady Elliot Island Airstrip,LYT,-24.112900,152.715600
11413,Gruyere Airport,GYZ,-28.034548,123.815060
11735,Agnew Airport,AGW,-12.145600,142.149369
11736,Alroy Downs Airport,AYD,-19.290800,136.078995
...,...,...,...,...
82466,Yam Island Airport,XMY,-9.899187,142.774265
82467,Yuendumu Airport,YUE,-22.254200,131.781998
82468,Young Airport,NGA,-34.255600,148.248001
82471,Yorketown Airport,ORR,-35.003564,137.619295


Then we can stick them together with `concat`. This is not (for reasons unclear) a `DataFrame` method, but a `pandas` function:

In [24]:
pd.concat([airports_nz, airports_au])

,name,lat,lon,iata_code
50060,Fox Glacier Aerodrome,-43.461110,170.017020,FGL
50124,Auckland International Airport,-37.011990,174.786331,AKL
50129,Taupo Airport,-38.739700,176.084000,TUO
50130,Ardmore Airport,-37.029701,174.973007,AMZ
50131,Ashburton Airport,-43.903301,171.796997,ASG
...,...,...,...,...
82466,Yam Island Airport,-9.899187,142.774265,XMY
82467,Yuendumu Airport,-22.254200,131.781998,YUE
82468,Young Airport,-34.255600,148.248001,NGA
82471,Yorketown Airport,-35.003564,137.619295,ORR


It's worth noting here that the Australian airport data columns are in a different order, but that `pandas` finds the matching names and lines them up.

If the column names in the two dataframes don't match then you get lots of NA values in the combined table because `pandas` is not psychic and doesn't know that two related names reference the same thing.

In [25]:
airports_au = airports_all[(airports_all.iso_country == "AU") &
                           (airports_all.iata_code.notna())]
airports_au = airports_au[
    ["name", "latitude_deg", "longitude_deg", "iata_code"]]
pd.concat([airports_nz, airports_au])

,name,lat,lon,iata_code,latitude_deg,longitude_deg
50060,Fox Glacier Aerodrome,-43.461110,170.017020,FGL,NaN,NaN
50124,Auckland International Airport,-37.011990,174.786331,AKL,NaN,NaN
50129,Taupo Airport,-38.739700,176.084000,TUO,NaN,NaN
50130,Ardmore Airport,-37.029701,174.973007,AMZ,NaN,NaN
50131,Ashburton Airport,-43.903301,171.796997,ASG,NaN,NaN
...,...,...,...,...,...,...
82466,Yam Island Airport,NaN,NaN,XMY,-9.899187,142.774265
82467,Yuendumu Airport,NaN,NaN,YUE,-22.254200,131.781998
82468,Young Airport,NaN,NaN,NGA,-34.255600,148.248001
82471,Yorketown Airport,NaN,NaN,ORR,-35.003564,137.619295


And things get even weirder if you concatenate column-wise.

In [26]:
pd.concat([airports_nz, airports_au], axis = "columns")

,name,lat,lon,iata_code,name,latitude_deg,longitude_deg,iata_code
50060,Fox Glacier Aerodrome,-43.461110,170.017020,FGL,NaN,NaN,NaN,NaN
50124,Auckland International Airport,-37.011990,174.786331,AKL,NaN,NaN,NaN,NaN
50129,Taupo Airport,-38.739700,176.084000,TUO,NaN,NaN,NaN,NaN
50130,Ardmore Airport,-37.029701,174.973007,AMZ,NaN,NaN,NaN,NaN
50131,Ashburton Airport,-43.903301,171.796997,ASG,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
82466,NaN,NaN,NaN,NaN,Yam Island Airport,-9.899187,142.774265,XMY
82467,NaN,NaN,NaN,NaN,Yuendumu Airport,-22.254200,131.781998,YUE
82468,NaN,NaN,NaN,NaN,Young Airport,-34.255600,148.248001,NGA
82471,NaN,NaN,NaN,NaN,Yorketown Airport,-35.003564,137.619295,ORR


In sum, while you can use `pd.concat` to assemble big tables from small ones, you need to do this with some care, making sure that the structure of the two (or more tables are compatible). The most likely use-case for this is recombining results of some tabular data processing applied to data one subgroup at a time. This is something we look at in the next notebook. 